# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [ ]:
!pip install chromadb langchain langchain_community langchain_openai

In [ ]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [ ]:
llm = OpenAI(temperature=0)

In [ ]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "hamlet.txt")

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving hamlet.txt to hamlet (1).txt


In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(
    texts,
    embeddings,
    collection_name="hamlet",
      persist_directory="chroma_db"  # Directory to store the Chroma database
    )

docsearch.persist()

<ipython-input-72-d9c413f4cb1b>:19: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  docsearch.persist()


In [ ]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

In [ ]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [ ]:
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(
    ruff_texts,
    embeddings,
    collection_name="ruff",
    persist_directory="chroma_ruff_db" )
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

## Create the Agent

In [ ]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [ ]:
tools = [
    Tool(
        name="Hamlet Character Analysis",
        func=state_of_union.run,
        description="Useful for answering questions about the characters in Shakespeare's Hamlet. Input should be a fully formed question.",
    ),
    Tool(
        name="Hamlet Plot Summary QA",
        func=ruff.run,
        description="Useful for answering questions about the plot and events in Hamlet. Input should be a fully formed question.",
    ),
]

In [ ]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent.invoke(
    "What does Hamlet say in his 'To be or not to be' soliloquy?"
)



> Entering new AgentExecutor chain...
 I should use the Hamlet Character Analysis tool to find out what Hamlet says in his soliloquy.
Action: Hamlet Character Analysis
Action Input: 'To be or not to be' soliloquy
Observation:  This is a famous soliloquy from Shakespeare's play Hamlet, in which the main character, Hamlet, contemplates the idea of life and death. He is struggling with the decision of whether it is better to continue living and facing the struggles of life, or to end his life and potentially find peace in death. He also considers the unknowns of death and the fear of what may come after. This soliloquy is often seen as a representation of the human struggle with mortality and the fear of the unknown.
Thought: I now have a better understanding of what Hamlet says in his soliloquy.
Final Answer: In his 'To be or not to be' soliloquy, Hamlet contemplates the idea of life and death and the fear of the unknown.

> Finished chain.


{'input': "What does Hamlet say in his 'To be or not to be' soliloquy?",
 'output': "In his 'To be or not to be' soliloquy, Hamlet contemplates the idea of life and death and the fear of the unknown."}

In [ ]:
agent.invoke("Why does Hamlet hesitate to take revenge on Claudius?")




> Entering new AgentExecutor chain...
 Hamlet's hesitation is a key aspect of his character and understanding it is crucial to understanding the play.
Action: Hamlet Character Analysis
Action Input: "Why does Hamlet hesitate to take revenge on Claudius?"
Observation:  Hamlet hesitates to take revenge on Claudius because he is envious of Laertes' skill with a rapier and wants to challenge him to a duel.
Thought: This is an interesting insight, but there may be more to it.
Action: Hamlet Plot Summary QA
Action Input: "Why does Hamlet hesitate to take revenge on Claudius?"
Observation:  Hamlet hesitates to take revenge on Claudius because he is unsure if the ghost of his father is truly his father's spirit or a demon trying to deceive him. He also struggles with the moral implications of taking revenge and the potential consequences of his actions. Additionally, Hamlet is a thoughtful and introspective character, and he wants to make sure that his actions are justified before he acts up

{'input': 'Why does Hamlet hesitate to take revenge on Claudius?',
 'output': "The main reasons for Hamlet's hesitation to take revenge on Claudius are his uncertainty about the ghost's identity, his moral and philosophical considerations, and his envy of Laertes' skill."}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [ ]:
tools = [
    Tool(
        name="Hamlet Character QA System",
        func=state_of_union.run,
        description="Useful for answering questions about the characters in Shakespeare's Hamlet. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Hamlet Plot QA System",
        func=ruff.run,
        description="Useful for answering questions about the plot, acts, and scenes of Hamlet. Input should be a fully formed question. Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [ ]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent.invoke(
 "What does Hamlet say about death in his 'To be or not to be' soliloquy?")



> Entering new AgentExecutor chain...
 I should use the Hamlet Character QA System to answer this question.
Action: Hamlet Character QA System
Action Input: "What does Hamlet say about death in his 'To be or not to be' soliloquy?"
Observation:  Hamlet contemplates the idea of death and whether it is better to suffer through life or to end it. He sees death as a way to escape the pain and struggles of life, but also acknowledges the uncertainty of what comes after death. He ultimately decides that the fear of the unknown after death is what keeps people from ending their lives.


> Finished chain.


{'input': "What does Hamlet say about death in his 'To be or not to be' soliloquy?",
 'output': ' Hamlet contemplates the idea of death and whether it is better to suffer through life or to end it. He sees death as a way to escape the pain and struggles of life, but also acknowledges the uncertainty of what comes after death. He ultimately decides that the fear of the unknown after death is what keeps people from ending their lives.'}

In [ ]:
agent.invoke("Why does Hamlet pretend to be mad?")



> Entering new AgentExecutor chain...
 This question is about the character of Hamlet, so I should use the Hamlet Character QA System.
Action: Hamlet Character QA System
Action Input: "Why does Hamlet pretend to be mad?"
Observation:  Hamlet pretends to be mad in order to deceive and manipulate those around him, particularly his uncle King Claudius, who he believes to be responsible for his father's death. He also uses his madness as a way to express his true feelings and frustrations without being held accountable for his actions.


> Finished chain.


{'input': 'Why does Hamlet pretend to be mad?',
 'output': " Hamlet pretends to be mad in order to deceive and manipulate those around him, particularly his uncle King Claudius, who he believes to be responsible for his father's death. He also uses his madness as a way to express his true feelings and frustrations without being held accountable for his actions."}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [ ]:
tools = [
    Tool(
        name="Hamlet Character QA System",
        func=state_of_union.run,
        description="Useful for answering questions about the characters in Shakespeare's Hamlet. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Hamlet Plot QA System",
        func=ruff.run,
        description="Useful for answering questions about the plot, acts, and scenes in Shakespeare's Hamlet. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [ ]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent.invoke(
    "How does Hamlet feel about Claudius in Act I, and does he express similar feelings later in the play?"
)



> Entering new AgentExecutor chain...
 I should use the Hamlet Character QA System to answer this question.
Action: Hamlet Character QA System
Action Input: "How does Hamlet feel about Claudius in Act I?"
Observation:  In Act I, Hamlet is suspicious of Claudius and believes that he may have had a hand in his father's death. He is also angry and resentful towards Claudius for marrying his mother so soon after his father's death.
Thought: I should also check if Hamlet expresses similar feelings later in the play.
Action: Hamlet Character QA System
Action Input: "Does Hamlet express similar feelings towards Claudius later in the play?"
Observation:  Yes, Hamlet continues to express negative feelings towards Claudius throughout the play, as he believes Claudius is responsible for his father's death and is determined to seek revenge.
Thought: I now know the final answer.
Final Answer: Yes, Hamlet feels suspicious, angry, and resentful towards Claudius in Act I and continues to express sim

{'input': 'How does Hamlet feel about Claudius in Act I, and does he express similar feelings later in the play?',
 'output': 'Yes, Hamlet feels suspicious, angry, and resentful towards Claudius in Act I and continues to express similar feelings towards him throughout the play.'}